# Day 3: Crossed Wires
## Part One
The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is `R8,U5,L5,D3`, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:
```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```
Then, if the second wire's path is `U7,R6,D4,L4`, it goes up 7, right 6, down 4, and left 4:
```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```
These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

- `R75,D30,R83,U83,L12,D49,R71,U7,L72` - `U62,R66,U55,R34,D71,R55,D58,R83` = distance 159
- `R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51` - `U98,R91,D20,R16,D67,R40,U7,R15,U6,R7` = distance 135

**What is the Manhattan distance** from the central port to the closest intersection?

In [1]:
import numpy as np

In [2]:
test_1 = "R8,U5,L5,D3\nU7,R6,D4,L4"
test_2 = "R75,D30,R83,U83,L12,D49,R71,U7,L72\nU62,R66,U55,R34,D71,R55,D58,R83"
test_3 = "R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51\nU98,R91,D20,R16,D67,R40,U7,R15,U6,R7"

In [3]:
dirs = {
    'R': np.array([1,0]),
    'L': np.array([-1,0]),
    'U': np.array([0,1]),
    'D': np.array([0,-1])
}

In [4]:
def step(instruction, cur_pos):
    fields = []
    count = int(instruction[1:])
    for _ in range(count):
        cur_pos = cur_pos + dirs[instruction[0]]
        fields.append(cur_pos)
    return cur_pos, fields

In [5]:
def get_points(paths):
    points = set()
    pos = np.array([0,0])
    for direction in paths:
        pos, fields = step(direction, pos)
        [points.add(f.tostring()) for f in fields]
    return points

In [6]:
def task_one(paths):
    p1,p2 = [p.split(',') for p in paths.split('\n')]
    candidates = get_points(p1).intersection(get_points(p2))
    
    closest = float("inf")
    for intersection in candidates:
        pos = np.frombuffer(intersection, dtype=int)
        dist = sum(np.abs(pos))
        if dist < closest:
            closest = dist
    return closest

In [7]:
print(task_one(test_1))
print(task_one(test_2))
print(task_one(test_3))

6
159
135


In [8]:
with open("input/3_1.txt", 'r') as f:
    input_1 = f.read().strip()

In [9]:
task_one(input_1)

2129

## Part Two
It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:
```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```
In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

```
R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83
```
= 610 steps

```
R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7
```
= 410 steps

What is the fewest combined steps the wires must take to reach an intersection?

In [10]:
def steps_to(target, directions):
    pos = np.array([0,0])
    steps = 0
    for direction in directions:
        pos, fields = step(direction, pos)
        try:
            steps += [sum(np.equal(target, field))==2 for field in fields].index(True) + 1
            break
        except:
            steps += len(fields)
    return steps

In [11]:
def task_two(paths):
    p1,p2 = [p.split(',') for p in paths.split('\n')]
    candidates = get_points(p1).intersection(get_points(p2))

    closest = float("inf")
    for intersection in candidates:
        target = np.frombuffer(intersection, dtype=int)
        total = steps_to(target, p1) + steps_to(target, p2)
        if total < closest:
            closest = total
    return closest

In [12]:
print(task_two(test_1))
print(task_two(test_2))
print(task_two(test_3))

30
610
410


In [13]:
%%time
task_two(input_1)

CPU times: user 18.7 s, sys: 11.3 ms, total: 18.7 s
Wall time: 18.7 s


134662

In [14]:
def get_all_steps(path):
    all_steps = []
    pos = np.array([0,0])
    for direction in path:
        pos, fields = step(direction, pos)
        all_steps.extend(fields)
    return all_steps
    

In [15]:
def num_steps(all_steps, target):
    return [np.array_equal(target, field) for field in all_steps].index(True) + 1

In [16]:
def task_two_fast(paths):
    p1,p2 = [p.split(',') for p in paths.split('\n')]
    candidates = get_points(p1).intersection(get_points(p2))

    steps_1 = get_all_steps(p1)
    steps_2 = get_all_steps(p2)
    closest = float("inf")
    
    for intersection in candidates:
        target = np.frombuffer(intersection, dtype=int)
        total = num_steps(steps_1, target) + num_steps(steps_2, target)
        if total < closest:
            closest = total
    return closest

In [17]:
%%time
task_two_fast(input_1)

CPU times: user 20.4 s, sys: 35.7 ms, total: 20.4 s
Wall time: 20.4 s


134662

In [18]:
paths = input_1
p1,p2 = [p.split(',') for p in paths.split('\n')]
candidates = get_points(p1).intersection(get_points(p2))

steps_1 = get_all_steps(p1)
steps_2 = get_all_steps(p2)

In [19]:
closest = float("inf")

for intersection in candidates:
    target = np.frombuffer(intersection, dtype=int)
    total = num_steps(steps_1, target) + num_steps(steps_2, target)
    if total < closest:
        closest = total
closest

134662

In [21]:
with open("input/3_1.txt", "r") as file:
	def crawl_wire():
		loc = [0, 0]
		steps = 0

		for move in file.readline().split(","):
			delta = {"L": (0, -1), "R": (0, 1), "U": (1, 1), "D": (1, -1)}[move[0]]

			for _ in range(int(move[1:])):
				loc[delta[0]] += delta[1]
				steps += 1

				yield tuple(loc), steps

	visited = {}

	for loc, steps in crawl_wire():
		if loc not in visited:
			visited[loc] = steps

	print(min(steps + visited[loc] for loc, steps in crawl_wire() if loc in visited))

134662
